In [1]:
import os
import threading
from pathlib import Path
from llama_index.core import Settings
from llama_index.core import StorageContext, load_index_from_storage
from llm_utils import get_default_settings, create_index, query_index, extract_ingredients_from_text, clean_string
from text2speech import text2speech, text2unreal_speech
from speech2text import listen, speech2text

pygame 2.5.2 (SDL 2.28.3, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
llm, embed_model, prompt_helper, node_parser = get_default_settings()

Settings.llm = llm
Settings.embed_model = embed_model
Settings.prompt_helper = prompt_helper
Settings.node_parser = node_parser

In [3]:
# Set filepaths
proj_dir = Path.cwd()
models_dir = proj_dir / 'models'
data_dir = proj_dir / 'data'

In [4]:
path_to_data = data_dir / 'pantry.txt'
path_to_cache = proj_dir / 'cache'

if os.listdir(path_to_cache) != 0:
    # If cache contains files then use it.
    storage_context = StorageContext.from_defaults(persist_dir=path_to_cache)
    index = load_index_from_storage(storage_context)
else:
  index = create_index(path_to_data, path_to_cache)

In [5]:
index = create_index(path_to_data, path_to_cache)

In [6]:
def llm_query():
    # Capture user query.
    audio = listen()
    user_query = speech2text(audio)

    if 'no thank you' in user_query.lower():
        return False

    # Play holding response whilst waiting for actual reponse from llm.
    thread = threading.Thread(target=text2unreal_speech, args=("One moment. Let me check for you.",))
    thread.start()
    response = query_index(index, user_query)
    thread.join()

    # Print and play llm response.
    response_text = response.response
    print('Answer: ', response_text)
    text2unreal_speech(response_text)

In [11]:
text2unreal_speech("Hi James, how can I help?")

llm_query()

again = True
while again:
    text2unreal_speech("Anything else I can help you with?")
    again = llm_query() 

text2unreal_speech("Ok, speak soon!")

Listening...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Text: do I have tomatoes in the pantry


Note: Illegal Audio-MPEG-Header 0x00fff3e0 at offset 36094.
Note: Trying to resync...
Note: Skipped 1 bytes in input.
Note: Illegal Audio-MPEG-Header 0x00fff3e0 at offset 36094.
Note: Trying to resync...
Note: Skipped 1 bytes in input.


Answer:   No, you don't have tomatoes in the pantry as it is mentioned "Not in stock: Tomato sauce".
Listening...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Text: no thank you
